In [1]:
import gffutils
import pandas as pd
import tqdm

In [4]:
gene_db = gffutils.FeatureDB('../genome/Homo_sapiens.GRCh38.96.db')
clip_db = pd.read_csv("../clip/clip_db.tsv", sep="\t")

In [3]:
clip_db.columns

Index(['sample', 'protein', 'chr', 'start', 'stop', 'strand',
       'log2(fold-enrichment)', '-log10(p-value)'],
      dtype='object')

In [4]:
clip_proteins = list(set(clip_db["protein"]))
print(len(clip_proteins))
clip_regions = [(f"{row['chr']}:{row['start']}-{row['stop']}", row["strand"]) for i, row in clip_db.iterrows()]
print(len(clip_regions))

130
425289


In [5]:
print(gene_db.schema())

CREATE TABLE features (
    id text,
    seqid text,
    source text,
    featuretype text,
    start int,
    end int,
    score text,
    strand text,
    frame text,
    attributes text,
    extra text,
    bin int,
    primary key (id)
    )
CREATE TABLE relations (
    parent text,
    child text,
    level int,
    primary key (parent, child, level)
    )
CREATE TABLE meta (
    dialect text,
    version text
    )
CREATE TABLE directives (
    directive text
    )
CREATE TABLE autoincrements (
    base text,
    n int,
    primary key (base)
    )
CREATE TABLE duplicates (
    idspecid text,
    newid text,
    primary key (newid)
    )
CREATE INDEX relationsparent ON relations (parent)
CREATE INDEX relationschild ON relations (child)
CREATE INDEX featuretype ON features (featuretype)
CREATE INDEX seqidstartend ON features (seqid, start, end)
CREATE INDEX seqidstartendstrand ON features (seqid, start, end, strand)
CREATE TABLE sqlite_stat1(tbl,idx,stat)


In [40]:
print(gene_db.execute(f'''
    SELECT attributes FROM features
    WHERE (featuretype=='transcript')
''').fetchone()[:])

('{"gene_id":["ENSG00000223972"],"gene_version":["5"],"transcript_id":["ENST00000456328"],"transcript_version":["2"],"gene_name":["DDX11L1"],"gene_source":["havana"],"gene_biotype":["transcribed_unprocessed_pseudogene"],"transcript_name":["DDX11L1-202"],"transcript_source":["havana"],"transcript_biotype":["processed_transcript"],"tag":["basic"],"transcript_support_level":["1"]}',)


In [10]:
list(gene_db.region(region="1:11869-14409", featuretype="transcript"))

[<Feature transcript (1:11869-14409[+]) at 0x7f7a60b08790>,
 <Feature transcript (1:12010-13670[+]) at 0x7f799f7bea60>,
 <Feature transcript (1:14404-29570[-]) at 0x7f799f7be7c0>]

In [54]:
list(gene_db.featuretypes())

['CDS',
 'Selenocysteine',
 'exon',
 'five_prime_utr',
 'gene',
 'start_codon',
 'stop_codon',
 'three_prime_utr',
 'transcript']

In [24]:
for exon in gene_db.all_features(featuretype="five_prime_utr"):
    print(exon)
    break
# for exon in gene_db.region(start=11869, end=12227, featuretype="exon"):
#     print(exon)

1	havana	five_prime_utr	65419	65433	.	+	.	gene_id "ENSG00000186092"; gene_version "6"; transcript_id "ENST00000641515"; transcript_version "2"; gene_name "OR4F5"; gene_source "ensembl_havana"; gene_biotype "protein_coding"; transcript_name "OR4F5-202"; transcript_source "havana"; transcript_biotype "protein_coding"; tag "basic";


In [9]:
transcripts = []
# tqderator = tqdm.tqdm(filter(lambda args: args[0] < 3,  enumerate(clip_regions)))
tqderator = tqdm.tqdm(clip_regions)
# for i, region in tqderator:
for region in tqderator:
    # for transcript in gene_db.region(region=region[0], featuretype="transcript", strand=region[1]):
    for transcript in gene_db.region(region=region[0], featuretype="exone", strand=region[1]):
        reg_start, reg_stop = (int(s) for s in region[0].split(":")[-1].split("-"))
        # if (reg_start < transcript.start) or (reg_stop > transcript.stop):
        #     continue
        transcripts.append((transcript.id, region))
        tqderator.set_description(len(transcripts))

  0%|          | 1670/425289 [01:32<6:31:24, 18.04it/s]


KeyboardInterrupt: 

In [47]:
len(transcripts)

10

In [9]:
print(gene_db.execute(f'''
    SELECT attributes FROM features
    WHERE (featuretype=='transcript')
''').fetchone()[:])

AttributeError: 'FeatureDB' object has no attribute 'id'

In [10]:
list(gene_db.all_features())

KeyboardInterrupt: 

In [19]:
for ft in gene_db.all_features(featuretype="transcript"):
    if ft.attributes["transcript_id"][0] == "ENST00000519980":
        print(ft)
# for exon in gene_db.region(start=11869, end=12227, featuretype="exon"):
#     print(exon)

8	ensembl_havana	transcript	140531421	140635546	.	-	.	gene_id "ENSG00000123908"; gene_version "12"; transcript_id "ENST00000519980"; transcript_version "5"; gene_name "AGO2"; gene_source "ensembl_havana"; gene_biotype "protein_coding"; transcript_name "AGO2-205"; transcript_source "ensembl_havana"; transcript_biotype "protein_coding"; tag "CCDS,basic"; ccds_id "CCDS55279"; transcript_support_level "1";


In [34]:
gene_db['1']

FeatureNotFoundError: 1

In [42]:
print(gene_db.execute(f'''
    SELECT id FROM features
''').fetchone()[:])

('CDS_1',)


In [47]:
for pr in gene_db.parents(gene_db["CDS_1"], featuretype="gene"):
    print(pr.id)

ENSG00000186092


In [56]:
for ex in gene_db.children(gene_db["ENST00000335137"], featuretype="exon"):
    print(ex.start, ex.end)

69055 70108


In [67]:
print(gene_db["ENST00000367178"])

6	ensembl_havana	transcript	154733325	154834060	.	+	.	gene_id "ENSG00000213079"; gene_version "9"; transcript_id "ENST00000367178"; transcript_version "7"; gene_name "SCAF8"; gene_source "ensembl_havana"; gene_biotype "protein_coding"; transcript_name "SCAF8-201"; transcript_source "ensembl_havana"; transcript_biotype "protein_coding"; tag "CCDS,basic"; ccds_id "CCDS5247"; transcript_support_level "2";
